In [1]:
#to access the data/files from the drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#PATHs: (don't forget the last '/')
##to load:
video3DName = '20190805_SJR3.2.2_w1_s2.nd2'
video3DPATH = '/content/drive/My Drive/gitRendu/data/videos/'
groundTruthPATH = '/content/drive/My Drive/gitRendu/data/annotations/'
idxNameMapPATH = '/content/drive/My Drive/gitRendu/data/annotations/'
##to save :
framesPATH = '/content/drive/My Drive/gitRendu/data/frames/'
masksPATH = '/content/drive/My Drive/gitRendu/data/masks/'

In [6]:
!pip install nd2reader

In [0]:
import numpy as np
import time
import glob, os
from nd2reader import ND2Reader
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot as plt
import skimage
import sys
from skimage.transform import resize
from scipy.ndimage import zoom
import cv2

# Functions

In [0]:
def prep_nd2_reader(reader, c=1):
    """
    Prepare the parameters of the ND2Reader object to be consistent with our data
    bundled_axes "yxz"
    default_cord for the channel of interest
    So looping over the reader goes stright through time
    """

    reader.default_coords['c'] = c
    reader.bundle_axes = 'yxz'


def get3DImage(images, time):
    #capture the nd2 3d image at a specific time
    #return shape : 312,512,35
    images.iter_axes = 't'
    image = images[time]

    return image


def getMaskAsArray(maskDf, maskShape, time, nameMapDf):
    #return the mask at a specific time as an array
    #return shape : 312,512,35
    
    #select the good time
    maskDfTimed = maskDf[maskDf.Time == time]    
    #find 'Noise' value (to remove them from the mask)
    noiseValue = nameMapDf[nameMapDf.name == 'Noise'].values.tolist()[0][1]
    #remove 'Noise' from mask (i.e. 'Noise' will have value 0)
    #########we decided to keep the noise values
    maskDfTimedWithoutNoise = maskDfTimed#maskDfTimed[maskDfTimed.Segment != noiseValue]
    #create outpute array
    array = np.zeros(maskShape)
    #insert mask values in array
    maskList = maskDfTimedWithoutNoise[['x', 'y', 'z', 'Segment']].values.tolist()
    #print(maskList)
    for elt in maskList:
        array[elt[0], elt[1], elt[2]] = elt[3]
    return array

def maskAllCells(mask, nameMap):
    #copy the mask
    result = mask.copy()
    #find the index of cells (except noise)
    nameMap = nameMap[nameMap.name != 'Noise']
    nameMap = nameMap[nameMap.name != 'noise']
    listIndex = nameMap[" name_idx"].tolist()
    #change the values of the mask
    result['Segment'] = result['Segment'].apply(lambda x: 1 if x in listIndex else 2)
    
    return result

def AIANoduleGroundTruth(ground_truth_df, nameMap):
    #return a new ground_truth with the value 1 for 'AIA_terminal_nodule', 2 for 'AIA_central_nodule' and 3 for 
    #all other cells parts. Also return a new 'idx_name_map' df
    
    #copy the dataframes
    resultGroundTruth = ground_truth_df.copy()
    resultNameMap = nameMap.copy()
    #give the groud_truth a new index called 'newName_idx' with value 3 by default
    resultNameMap['newName_idx'] = 3
    resultNameMap.loc[resultNameMap.name == 'AIA_terminal_nodule', 'newName_idx'] = 1
    resultNameMap.loc[resultNameMap.name == 'AIA_central_nodule', 'newName_idx'] = 2
    #create a dictionary to map from old index to new index on the mask
    dictionary = dict(zip(resultNameMap[' name_idx'], resultNameMap['newName_idx']))
    #map ground_truth Segment value from old to new index
    resultGroundTruth['Segment'] = resultGroundTruth['Segment'].map(dictionary)
    #create the corresponding 'idx_name_map' df
    resultNameMap.loc[resultNameMap.newName_idx == 3, 'name'] = 'Noise'
    resultNameMap = resultNameMap[['name', 'newName_idx']].drop_duplicates().rename(columns={'newName_idx': ' name_idx'})\
                                                                                   .sort_values(by=[' name_idx']).reset_index(drop=True)
    return resultGroundTruth, resultNameMap

# Generating the Frames & Masks 

In [0]:
gt_df = pd.read_csv(groundTruthPATH + video3DName[:-4] + '_ground_truth.csv')
nameMap_df = pd.read_csv(idxNameMapPATH + video3DName[:-4] + '_idx_name_map.csv')
realnd2FileName = video3DPATH + video3DName

###shape of frame & mask
f_shape = (80, 128, 35)
m_shape = (80, 128, 35)

print(video3DName[:-4] + '_ground_truth.csv')
print(video3DName[:-4] + '_idx_name_map.csv')
print(video3DName)

with ND2Reader(realnd2FileName) as images:

    images.iter_axes = 't'
    prep_nd2_reader(images)
    i = 0
    for image in images:
        
        print("Loading frame ",i)
        fp = "frame_"+str(int(i))+".npy"
        mp = "mask_"+str(int(i))+".npy"
        
        #the DenseNetFCN-3D need a precise input shape
        data_res = resize(image, f_shape)        
        np.save(framesPATH + fp, data_res[:,:,:32])
        
        resultGroundTruth, resultNameMap =  AIANoduleGroundTruth(gt_df, nameMap_df)
        mask = getMaskAsArray(resultGroundTruth, image.shape, i, resultNameMap)
        mask_res = resize(mask, m_shape, mode='edge',anti_aliasing=False,anti_aliasing_sigma=None, order=0)

        np.save(masksPATH + mp, mask_res[:,:,:32])

        i = i + 1
print("######################")

20190805_SJR3.2.2_w1_s2_ground_truth.csv
20190805_SJR3.2.2_w1_s2_idx_name_map.csv
20190805_SJR3.2.2_w1_s2.nd2
Loading frame  0
Loading frame  1
Loading frame  2
Loading frame  3
Loading frame  4


KeyboardInterrupt: 